### Part 2. Visualizing any coordinate file using NGLView

In [8]:
def visualize_file(file_name, representation_type):
    """
    Uses NGLView to render any coordinate file belonging to the CELL. 

    :param filename: str, name of the file containing the molecular structure
    :param representation: str, representation to display (default='spacefill,surface')
    """
    # Read the file and extract the atom names
    with open(file_name, "r") as f:
        lines = f.readlines()[2:-1]
        atom_names = [line[10:15].strip() for line in lines]

    # Get the indices of the different atom types. Names are currently hardcoded
    type_A = [i for i, name in enumerate(atom_names) if name == "A"]
    type_B = [i for i, name in enumerate(atom_names) if name == "B"]
    type_N = [i for i, name in enumerate(atom_names) if name == "N"]

    view = nv.show_file(file_name, default=False)
    view.center()
    view.clear_representations()

    #there are two representation types that make sense, spacefilling (spheres) and surface
    if representation_type == 'spacefilling':
        # Represent the atoms as beads without bonds
        view.add_spacefill(selection=type_A, color='#0047AB', radius_type='vdw', radius=0.8, opacity=1) #blue
        view.add_spacefill(selection=type_B, color='#A30000', radius_type='vdw', radius=0.8, opacity=1) #red
        view.add_spacefill(selection=type_N, color='#006400', radius_type='vdw', radius=0.8, opacity=1) #green
    elif representation_type == 'surface':
        # Alternatively, represent as surface representation
        view.add_surface(selection=type_A, color='#0047AB', opacity=0.9) #blue
        view.add_surface(selection=type_B, color='#A30000', opacity=0.9) #red
        view.add_surface(selection=type_N, color='#006400', opacity=0.9) #green
    else:
        raise ValueError('Representation type must be "surface" or "spacefilling"')
    
    #some NGLview settings that make it nicer
    view.camera = 'orthographic'
    view.layout.width = 'auto'
    view._remote_call("setSize", target="Widget", args=["500px", "500px"])
    
    return view
        
view = visualize_file('CELL.gro', 'surface')
#view = visualize_file('CELL.gro', 'spacefilling')
view


NGLWidget(layout=Layout(width='auto'))

### Part 3. Using the generated coordinates and topologies to start a GMX simulation

### Generating .mdp inputs and spawning simulations

In [20]:
#since we are working with dicts, we can change mdp settings simply through their key
#run_mdp['nsteps'] = 1000
write_mdp_file(min_mdp, 'min.mdp')
write_mdp_file(run_mdp, 'run.mdp')
run_GMX()


Simulation complete, no problems encountered. Log saved as "gmx_run.log"


In [12]:
#Visualizing the final frame of the simulation
view = visualize_file('2-run.gro', 'spacefilling')
view


NGLWidget(layout=Layout(width='auto'))

### Packing multiple cells using gmx editconf

In [23]:
"""import subprocess as sp

def generate_coordinates(box_size="6 6 6"):
    GMX = find_executable("gmx")
    #raise an eror if gmx cannot be found on the user's machine
    if not GMX:
        raise RuntimeError("Cannot find GROMACS executable 'gmx' in PATH")
    positions = ["0", "1.9", "-1.9"]
    combos = [(x, y, z) for x in positions for y in positions for z in positions]
    for i, combo in enumerate(combos):
        x, y, z = combo
        cmd = f"gmx editconf -f CELL_OG.gro -o translate{i+1}.gro -translate {x} {y} {z}"
        sp.run(cmd.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)"""
        


### Spawning US simulations through Python

In [8]:
"""def make_ndx_pull(atom_indices_dict=None, gro="CELL.gro", ndx = "index_pull.ndx"):
    
    if atom_indices_dict is None:
        atom_indices_list = list(range(1, 21))
        atom_range = "a" + str(atom_indices_list[0]) + "-" + str(atom_indices_list[-1])
    else:
        atom_indices_list = []
        for start, end in atom_indices_dict.values():
            atom_indices_list += list(range(start, end+1))

    # Define the command for creating the index file and adding a new group
    command = f"gmx make_ndx -f {gro} -o {ndx} <<EOF\na 1-20\nname 3 TO_PULL\nq\nEOF"

    # Suppress all output from the command
    with open(os.devnull, "w") as fnull:
        sp.call(command, shell=True, stdout=fnull, stderr=fnull)
        """
#create_index_file()

'def make_ndx_pull(atom_indices_dict=None, gro="CELL.gro", ndx = "index_pull.ndx"):\n    \n    if atom_indices_dict is None:\n        atom_indices_list = list(range(1, 21))\n        atom_range = "a" + str(atom_indices_list[0]) + "-" + str(atom_indices_list[-1])\n    else:\n        atom_indices_list = []\n        for start, end in atom_indices_dict.values():\n            atom_indices_list += list(range(start, end+1))\n\n    # Define the command for creating the index file and adding a new group\n    command = f"gmx make_ndx -f {gro} -o {ndx} <<EOF\na 1-20\nname 3 TO_PULL\nq\nEOF"\n\n    # Suppress all output from the command\n    with open(os.devnull, "w") as fnull:\n        sp.call(command, shell=True, stdout=fnull, stderr=fnull)\n        '

### Finding bond radiuses with lowest energy

In [25]:
import os
import shutil
import numpy as np
import pandas as pd
from itertools import product

working_dir = "/wrk/matspunt/projects/cell_model/param_C180/energy_radius_search"

def get_iterated_path(fname_path):
    """
    Get the path to a filename which does not exist by incrementing path.
    """
    if not os.path.exists(fname_path):
        return fname_path
    filename, file_extension = os.path.splitext(fname_path)
    i = 1
    new_fname = "{}-{}{}".format(filename, i, file_extension)
    while os.path.exists(new_fname):
        i += 1
        new_fname = "{}-{}{}".format(filename, i, file_extension)
    return new_fname

r0_range = np.arange(1.2, 2.5 , 0.05)

os.chdir(working_dir)

#create dataframe with all param combinations
params = pd.DataFrame(list(product(r0_range)),columns=['r0_range'])

print("\n\nnr of param combs: " + str(len(params))) #to get some idea of number of param combinations)

for index, row in params.iterrows():
    dirname = get_iterated_path("sim")
    print(dirname) #let's print the dirname so we have some idea of the progress
    shutil.copytree(r"sim", dirname)
    os.chdir(dirname)
    os.chdir("toppar")
    with open('CELL.itp', 'r') as f:
        lines = f.readlines()

    with open('CELL.itp', 'w') as f:
        for line in lines:
            line = line.replace('N_A_r0', '{}'.format(str(row['r0_range'])))
            line = line.replace('N_B_r0', '{}'.format(str(row['r0_range'])))
            f.write(line)
            
    os.chdir("../")
    os.system("gmx grompp -p system.top -f toppar/min.mdp -c CELL.gro -o 1-min -maxwarn 1 >/dev/null 2>&1")
    os.system("timeout -k 2s 30s gmx mdrun -nt 12 -pin on -deffnm 1-min >/dev/null 2>&1")

    os.system("gmx grompp -p system.top -f toppar/run.mdp -c 1-min.gro -o 2-run -maxwarn 1 >/dev/null 2>&1" )
    os.system("timeout -k 2s 120s gmx mdrun -nt 12 -pin on -deffnm 2-run >/dev/null 2>&1")
    try:
        with open("2-run.log") as f, open("data.log", "w") as out_file:
            for line in f:
                if "Writing checkpoint, step 350000" in line:
                    out_file.write(line)
                    for i in range(8):
                        out_file.write(next(f))
                    break
    except FileNotFoundError:
        print("The file '2-run.log' was not found.")
    try:
        with open("toppar/CELL.itp") as f, open("data.log", "a") as out_file:
            for line in f:
                if "1  181 1" in line:
                    out_file.write("r0" + line)
                    break
    except FileNotFoundError:
        print("The file 'toppar/CELL.itp' was not found.")
    os.chdir(working_dir)




nr of param combs: 26
sim-1
The file '2-run.log' was not found.
sim-2
The file '2-run.log' was not found.
sim-3
The file '2-run.log' was not found.
sim-4
The file '2-run.log' was not found.
sim-5
The file '2-run.log' was not found.
sim-6
The file '2-run.log' was not found.
sim-7
The file '2-run.log' was not found.
sim-8
The file '2-run.log' was not found.
sim-9
The file '2-run.log' was not found.
sim-10
The file '2-run.log' was not found.
sim-11
The file '2-run.log' was not found.
sim-12
The file '2-run.log' was not found.
sim-13
The file '2-run.log' was not found.
sim-14
The file '2-run.log' was not found.
sim-15
The file '2-run.log' was not found.
sim-16
The file '2-run.log' was not found.
sim-17
The file '2-run.log' was not found.
sim-18
The file '2-run.log' was not found.
sim-19
The file '2-run.log' was not found.
sim-20
The file '2-run.log' was not found.
sim-21
The file '2-run.log' was not found.
sim-22
The file '2-run.log' was not found.
sim-23
The file '2-run.log' was not fou

In [2]:
text = '''[ bondtypes]
;  i   j  func    r0      fk   
   C   M1   1     1.60    5000
   C   M2   1     1.80    10000'''

lines = text.strip().split('\n')[2:]  # Remove the first two lines and split the remaining lines
bondtypes_dict = {}

for line in lines:
    values = line.split()
    i, j, func, r0, fk = values[0], values[1], int(values[2]), float(values[3]), int(values[4])
    bondtypes_dict[(i, j)] = {'func': func, 'r0': r0, 'fk': fk}

print(bondtypes_dict)

{('C', 'M1'): {'func': 1, 'r0': 1.6, 'fk': 5000}, ('C', 'M2'): {'func': 1, 'r0': 1.8, 'fk': 10000}}
